In [ ]:
#This fitting utilized the package from manoharan-lab  to calculate the particle form factor. URL:https://github.com/manoharan-lab/manoharan-lab

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.optimize import least_squares
from scipy.optimize import curve_fit
import sys
sys.path.append(r'E:\repositories\python-mie-master')
sys.path.append( r'E:\repositories\structural-color-master')
import pymie as pm
from pymie import mie
import structcol as sc
import seaborn as sns
sns.set_style('white')
# For Jupyter notebooks only:
%matplotlib inline

In [ ]:
# set film parameters
wavelen = sc.Quantity('632.8 nm')  # laser wavelenth nm
particle_radius = sc.Quantity('520 nm')  # particle radius nm
n_particle = sc.Quantity(1.45,'')  # refractive index of particle
n_matrix = sc.Quantity(1.49,'')   # refractive index of matrix
n_eff = sc.Quantity(1.5,'')   # effective refractive index of matrix


In [ ]:

# input data
folder_path = r'F:\experiment\laser scattering system\test'
file_name ='output_file_250222.xlsx' #data after smooth
file_path = fr'{folder_path}\{file_name}'
sheet_name = "Sheet3"
C_qr=1 #column of qR
C_I_qR=2 #column of I_qR
qr_min = 1.5
qr_max = 4.0 

df = pd.read_excel(file_path, sheet_name=sheet_name, header=None)
df = df.dropna()  # Remove rows containing null values
qR = df[C_qr-1].values  # 
I_qR = df[C_I_qR-1].values  #
start_index = np.searchsorted(qR, qr_min)
end_index = np.searchsorted(qR, qr_max, side='right')
qR = df[0].iloc[start_index:end_index].to_numpy()
I_qR = df[1].iloc[start_index:end_index].to_numpy()


In [ ]:
# define form factor
def P_qR(qR,n_eff,wavelen, n_matrix, particle_radius):
    q = qR/particle_radius
    angles = np.arcsin(q*wavelen/(4*np.pi*n_eff))
    x = pm.size_parameter(wavelen, n_matrix, particle_radius)
    m = pm.index_ratio(n_particle, n_matrix)
    ipar, iperp = mie.calc_ang_dist(m, x, angles)
    P_qR = (ipar + iperp)/2
    return P_qR

# define structure factor
def S_qR(qR, l, phi, particle_radius):
    a_p = particle_radius.to('nm').magnitude  
    a_e = a_p + l   
    phi_e = phi * (a_e / a_p) ** 3  # effective volume fraction for hard spheres with an exluded annulus
    q=qR/a_p
    qa2 = 2 * q * a_e  # twice qa, for efficiency/conciseness
    lambda_1 = (1 + 2 * phi_e) ** 2 / (1 - phi_e) ** 4
    lambda_2 = -(1 + phi_e / 2) ** 2 / (1 - phi_e) ** 4
    c_1 = lambda_1 / qa2 ** 3 * (np.sin(qa2) - qa2 * np.cos(qa2))
    c_2 = -6 * phi_e * lambda_2 / qa2 ** 4 * (qa2 ** 2 * np.cos(qa2) - 2 * qa2 * np.sin(qa2) - 2 * np.cos(qa2) + 2)
    c_3 = -phi_e * lambda_1 / 2 / qa2 ** 6 * \
        (qa2 ** 4 * np.cos(qa2) - 4 * qa2 ** 3 * np.sin(qa2) - 12 * qa2 ** 2 * np.cos(qa2)
         + 24 * qa2 * np.sin(qa2) + 24 * np.cos(qa2) - 24)
    s_q = np.divide(1, 1 + 24 * phi_e * (c_1 + c_2 + c_3))
    return s_q

In [ ]:
%%time

# define fitting model for hardsphere

def model(qR, beta, phi, gamma):
    return beta * P_qR(qR,n_eff,wavelen, n_matrix, particle_radius) * S_qR(qR, 0, phi, particle_radius) + gamma

# set initial guess and bounds
initial_guess = [1, 0.2, 0]
bounds = ([0, 0.1, 0], [np.inf, 0.8, np.inf])
popt, pcov = curve_fit(model, qR, I_qR, p0=initial_guess, bounds=bounds)
beta_fitted, phi_fitted, gamma_fitted = popt

# print resluts
print("fitted beta:", beta_fitted)
print("fitted phi:", phi_fitted)
print("fitted gamma:", gamma_fitted)

# calculate R² and RMSE
I_fitted = model(qR, beta_fitted, phi_fitted, gamma_fitted)
ss_res = np.sum((I_qR - I_fitted) ** 2)
ss_tot = np.sum((I_qR - np.mean(I_qR)) ** 2)
r_squared = 1 - (ss_res / ss_tot)  
rmse = np.sqrt(ss_res / len(I_qR)) 
print(f"R²: {r_squared:.4f}")
print(f"RMSE: {rmse:.4f}")

# plot
plt.figure(figsize=(8, 6))
plt.plot(qR, I_qR, 'o', label='Original Data', markersize=5)
plt.plot(qR, I_fitted, '-', label='Fitted Curve', linewidth=2)
plt.xlabel('qR')
plt.ylabel('I(qR)')
plt.title('Fitting I(qR) = β * P(qR) * S(qR, φ) + γ')
plt.legend()
plt.show()

In [ ]:
%%time

# define fitting model for excluded annulus
def model(qR, beta, l, phi, gamma):
    return beta * P_qR(qR,n_eff,wavelen, n_matrix, particle_radius) * S_qR(qR, l, phi, particle_radius) + gamma

# set initial guess and bounds
initial_guess = [1, 200, 0.2, 0]
bounds = ([0,0, 0.1, 0], [np.inf, 700, 0.8, np.inf])
popt, pcov = curve_fit(model, qR, I_qR, p0=initial_guess, bounds=bounds)
beta_fitted, l_fitted, phi_fitted, gamma_fitted = popt

# print resluts
print("fitted beta:", beta_fitted)
print("fitted l:", l_fitted, "nm")
print("fitted phi:", phi_fitted)
print("fitted gamma:", gamma_fitted)

# calculate R² and RMSE
I_fitted = model(qR, beta_fitted, l_fitted, phi_fitted, gamma_fitted)
ss_res = np.sum((I_qR - I_fitted) ** 2)
ss_tot = np.sum((I_qR - np.mean(I_qR)) ** 2)
r_squared = 1 - (ss_res / ss_tot)  
rmse = np.sqrt(ss_res / len(I_qR)) 

print(f"R²: {r_squared:.4f}")
print(f"RMSE: {rmse:.4f}")

# plot
plt.figure(figsize=(8, 6))
plt.plot(qR, I_qR, 'o', label='Original Data', markersize=5)
plt.plot(qR, I_fitted, '-', label='Fitted Curve', linewidth=2)
plt.xlabel('qR')
plt.ylabel('I(qR)')
plt.title('Fitting I(qR) = β * P(qR) * S(qR, l, φ) + γ')
plt.legend()
plt.show()

In [ ]:
# Save results to an Excel file
output_df = pd.DataFrame({'qR': qR, 'origin':I_qR, 'I_fitted': I_fitted})
output_file_name ='output_file_fitted.xlsx' 
output_file_path = fr'{folder_path}\{output_file_name}'
output_df.to_excel(output_file_path, index=False)